<a href="https://colab.research.google.com/github/blue442/DS875/blob/main/fastapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We'll start by importing the required packages to the current environment

In [ ]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import requests
import uvicorn
from multiprocess import Process

Instantiate a '[FastAPI](https://fastapi.tiangolo.com/)' app object. FastAPI is a great library developed for python that allows you to easily deploy APIs with a few lines of code. Not only is it simple, but it has a number of useful features, and it's also quite scalable. This 'app' object will be the server logic that receives requests, then packages and returns responses using logic we will specify.

In [ ]:
app = FastAPI()

Now that we have an instance of the app, we'll create a route defined by a 'routing function'. The url of our route will be of the format [base_url]/[route]. The [base_url] will  be generated later by our 'ngrok' process, but we will specify the [route] information in the next cell with the line:


```
@app.get('[route]')
```

This first line is a python '[decorator](https://realpython.com/primer-on-python-decorators/)' - basically passing a function to a function. It basically tells our 'app' object (from above) that we want a 'get' request to the [route] to trigger the function defined below.


The rest of the code in the cell below initializes a function named 'home' with the line:

```
def home():
```

and then specifies the logic that will be performed by that function - in this case just return the string "Hello World":

```
return "Hello DS875!"
```


In [ ]:
@app.get('/index')
def home():
  return "Hello DS875!"

---
Now we're going to define a function that runs our FastAPI server so that it will produce responses to the requests at the route we specified. 

**EXTRA INFO:** There is a little 'magic' in this next function:
* We are deploying our server logic (the `app` object) using a service called '[uvicorn](https://www.uvicorn.org/)'. While `app` contains the logic of what we want to do, uvicorn actually implements it.
* We are using [ngrok](https://ngrok.com/), which is a service to connect the server running inside our notebook environment to the outside world. 
* We are also using a library called 'Process' which lets us push the webserver to the background, or the cell that runs it would be stuck.

In [ ]:
def run_api_server(app):
  ngrok_tunnel = ngrok.connect(8000)
  print('[base_url]:', ngrok_tunnel.public_url)
  base_url = ngrok_tunnel.public_url
  nest_asyncio.apply()
  server_process = Process(target = uvicorn.run, args=(app,), kwargs={'port': 8000})
  server_process.start()
  return server_process, base_url

We run that function, and it returns a process object (so we have a handle on the server running in the background) and the base url of our api server.

In [ ]:
server_process, base_url = run_api_server(app)

If we take the [base_url] as defined above, and append it with the 
[route] information from our routing function in the format [base_url]/[route] we should get the response "Hello DS875!". We'll do it with the requests module in the following cell, but you can also do it in a separate browser window, as we've just deployed a live API endpoint!

In [ ]:
requests.get(base_url + '/index').json()

We can generate a hyperlinked URL that should open in a new browser tab when clicked on:

In [ ]:
print(base_url + '/index')

And now you have your very own API service running! Note that this is extremely temporary, and can go down at any moment -  but should persist at least through this lab.

---

Now, to make it a little more exciting, we're going to add a more interactive route to our webserver so we can integrate some input. We'll have to start by stopping the webserver that is currently running so we can update it. The following cell will reconnect to the server if it is still running (using the `.join()` method) and close it (by calling the `.close()` method).

<font color=red>NOTE:</font> Becuase the `.join()` connects to a running process, you have to STOP the cell before running the `.close()` function. Depending on where the notebook is deployed, there are a number of ways to do this - click on the 'stop' symbol above or to the right of the cell, type 'i i' in quick succession, or go to the top menu and click on 'Kernel >> Interrupt'. Hopefully one of those will get you there.

In [ ]:
server_process.join()

We'll call the `.close()` function on the process, but it's likely closed from the interrupt - so this next cell may throw an error. No big deal.

In [ ]:
server_process.close()

Then kill the ngrok process:

In [ ]:
ngrok.kill()

Just like before, we'll add a new route to our app using a 'get' method - but this time, the request will include an 'id' value called 'item_id':

```
@app.get("/items/{item_id}")
```

In the second line, we'll define the function called 'read_item', which will take as input the value of 'item_id' specified in the url of the request:

```
def read_item(item_id: int):
```

Notice that we can force the 'item_id' to be an integer or 'int'). This is a nice built-in type checking provided by the FastAPI library.

Lastly, we want this function to package up the server's response - in this case we are just echoing the value of 'item_id' within a sentance, but the logic you implement here could be anything - e.g. querying for item information from a database.

```
return {"the item_id you supplied is ": item_id}
```


In [ ]:
@app.get("/items/{item_id}")
def read_item(item_id: int):
    return {"The item_id you supplied is": item_id}

Then we'll fire up the app again using the `run_api_server()` function:

In [ ]:
server_process, base_url = run_api_server(app)

Note that we likely have a different [base_url] now - ngrok randomly generates  a URL to connect our process to, but in a real API deployment this would be a static URL.

Now we can submit a request to that new url - in the example below, I've specified the item number to be '10', but you can change it to whatever you want.

In [ ]:
item_number = 10
requests.get(base_url + '/items/' + str(item_number)).json()

If you remember, we implemented some basic data type checking in the `read_items()` function that forced it to only allow integer values. What happens if we try to submit a string?

In [ ]:
item_number = 'malicious code!'
req = requests.get(base_url + '/items/' + item_number)
print('The response is: ' + str(req.json()))
print('The status code of the response is: ' + str(req.status_code))

As you can see, it rejected our request with a status code of '422'. This code means:

> **422 UNPROCESSABLE ENTITY**
> 
> The server understands the content type of the request entity (hence a 415 Unsupported Media Type status code is inappropriate), and the syntax of the request entity is correct (thus a 400 Bad Request status code is inappropriate) but was unable to process the contained instructions.

While this could be a mistake by the API user, this is also a basic way of protecting against malicious code being sent over to the server.




---

The last thing that we'll explore, is the built-in tooling provided by the FastAPI library to produce documentation. The beauty here is that our `app` has already put it together using the OpenAPI standard, and has made it available at a special route called  `/docs` -  the full URL is generated in the next cell:

In [ ]:
# generate an URL
url = base_url + '/docs'
print(url)

Finally, we want to re-attach to the server process that we pushed to the background, and stop it from running (again, you have to run the cell, stop it, and run it again :/)

In [ ]:
server_process.join()
server_process.close()

Finally, kill the ngrok pipe:

In [ ]:
ngrok.kill()